### 在函数上添加包装器
在包导入的时候装饰器就会执行.原来的函数命名会存储被装饰的函数func=wrapper(func)

### 创建装饰器时保留函数元信息
利用functools.wrap装饰器来注解底层包装函数

> @wrap还有一个重要特征可以通过__wrapped__直接访问被包装的函数.它还可以被装饰函数正确暴露底层的参数签名信息

In [11]:
from functools import wraps
import time
def timeit_func(func):

    @wraps(func) # 保留原函数信息
    def inner(*args,**kwargs): # 强调装饰器不会修改原始函数签名和返回值
        start=time.time()
        ret=func(*args,**kwargs)
        print('exec ',time.time()-start)
        return ret
    return inner

@timeit_func
def countdown(n):
    while n >0:
        n-=1
    
countdown(10)

print(countdown.__wrapped__)
from inspect import signature
print(signature(countdown)) # 暴露签名信息

exec  2.1457672119140625e-06
<function countdown at 0x103bdfe20>
(n)


### 解除一个装饰器
假设装饰器通过wrap来实现的,可以通过访问__wrapped__属性来访问原始函数

> 如果有多个装饰器,采用__wrapped__属性行为可能不可预知

### 带参数装饰器
9.4,9.5

In [17]:
from functools import wraps

def decorator1(func):
    @wraps(func)
    def wrapper(*args,**kwargs):
        func(*args,**kwargs)
        print('decorator1')
    return wrapper

def decorator2(func):
    @wraps(func)
    def wrapper(*args,**kwargs):
        func(*args,**kwargs)
        print('decorator2')
    return wrapper

@decorator2
@decorator1
def foo():
    ...

foo.__wrapped__()

decorator1


### 可选参数装饰器
在装饰器上加了参数要注意其写法.必须在装饰器后加括号.
```python
    @logger(level=INFO) 
    def func():
        ...
    
    # 等价于
    func=logger(level)(func)
```

### 利用装饰器强制函数上的类型检查

In [19]:
from inspect import signature
def type_check(*ty_args,**k_ty_args):
    def decorator(func):
       sig=signature(func)
       # 获取argument类型
       bound_type=sig.bind_partial(*ty_args,**k_ty_args).arguments
       @wraps(func)
       def wrapper(*args,**kwargs):
            bound_value=sig.bind(*args,**kwargs)
            for name,value in bound_value.arguments.items():
                if  name in bound_type:
                    if not isinstance(value,bound_type[name]):
                        raise TypeError('type must be',bound_type)
            return func(*args,**kwargs)
       return wrapper
    return decorator

@type_check(int,int)
def add(x,y):
    return x+y

add(1,2)

{'x': <class 'int'>, 'y': <class 'int'>}


3

### 将装饰器定义为类的部分
类上的装饰器看上去很奇怪,但是在标准库上却有很多比如@property装饰器

In [24]:
class A:

    def instance_wrapper(self,func):
        def wrapper(*args,**kwargs):
            print('instance...')
            return func(*args,**kwargs)
        return wrapper

    @classmethod
    def static_wrapper(cls,func):
        def wrapper(*args,**kwargs):
            print('static...')
            return func(*args,**kwargs)
        return wrapper
  
@A.static_wrapper
def foo():
    ...
a=A()
foo()

@a.instance_wrapper
def foo():
    ...
foo()

static...
instance...


### 将装饰器定义为类
